# Algoritmo Genético para treinamento da RNA

Importe os pacotes necessários

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras import optimizers
import numpy as np
from numpy import array, dtype
import argparse
import random
import pickle
import cv2
import os
import livelossplot
import collections
import asyncio
import sys


c:\users\johnp\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Estrutura do Algoritmo Genético

In [2]:
def inserePesosConv2d(lista, j, k, h):
	qtdPesos = 32
	g = 0
	while(g < qtdPesos):
		peso = random.uniform(-1.0, 1.0)
		lista[j][k][h].append(peso)
		g += 1
	return lista

def montaConv2d_1():
	camada = list()
	lista = list() ## no primeiro caso deve ter 3 posições, no segundo caso deve ter 32 posições

	descendente = 2 ## quantidade de elementos na camada

	i = 0
	while(i < descendente):
		lista = []
		if( i == 0):
			tamanhoSubCamada = 3
			j = 0
			while(j < tamanhoSubCamada):
				subLista = list()
				lista.append(subLista)

				tamanhoSubSubCamada = 3
				k = 0
				while(k < tamanhoSubSubCamada):
					subsubLista = list()
					lista[j].append(subsubLista)

					tamanhoSubSubSubcamada = 3
					h = 0
					while(h < tamanhoSubSubSubcamada):
						subsubsubLista = list()
						lista[j][k].append(subsubsubLista)

						lista = inserePesosConv2d(lista, j, k, h)

						h += 1
					k += 1
				j += 1
						
		elif(i == 1):
			qtdPesos = 32
			j = 0
			while(j < qtdPesos):
				lista.append(0.0)
				j += 1
		else:
			print("Não pode ter mais que duas posições")
		
		camada.append(np.array(lista, dtype='float32'))
		i += 1
	return camada



def montaCamada(arraysNaCamada, qtdPorArray):
	camada = list()
	return camada 

def montaBatch_normalization(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = list()
			
			pesoAtual = 0
			while(pesoAtual < qtdPorArray[i]):
				if(i == 0 or i == 3):
					numero = 1.0
				else:
					numero = 0.0

				lista.append(numero)
				pesoAtual += 1

			camada.append(np.array(lista, dtype='float32'))
			i = i + 1

	return camada 

def montaPesoNaDensa(lista):
	qtdPesos = 32768
	g = 0
	while(g < qtdPesos):
		sub = []
		peso = random.uniform(-1.0, 1.0)
		sub.append(peso)
		lista.append(sub)
		g += 1
	return lista

def montaDensa(arraysNaCamada, qtdPorArray):
	camada = list()
	if(not arraysNaCamada == 0):
		i = 0
		lista = None
		
		while(i < arraysNaCamada):
			lista = []	
			DEFAULT = 0.0
			if(i == 0):
				result = montaPesoNaDensa(lista)
			elif(i == 1):
				peso = DEFAULT
				lista.append(peso)
			else:
				print("Nao era pra cair aqui")

			i = i + 1
			camada.append(np.array(lista, dtype='float32'))
	
	return camada 



arraysNaCamada = None
qtdPorArray = None

array00 = np.array([],dtype='float32')

array0 = montaConv2d_1() 

array1 = montaCamada(0, []) # é vazio

array2 = montaBatch_normalization(4, [32,32,32,32])

array3 = montaCamada(0, []) # é vazio

array4 = montaCamada(0, []) # é vazio

array5 = montaDensa(2, [32768, 1])

array6 = montaBatch_normalization(4, [1,1,1,1])


print("conv2d_1 (Conv2D): ")
print(array0)

print("activation_1 (Activation): ")
print(array1)

print("batch_normalization_1 (Batch: ")
print(array2)

print("max_pooling2d_1 (MaxPooling2: " + ''.join(array3))
print(array3)

print("flatten_1 (Flatten): "+ ''.join(array4))
print(array4)

print("dense_1 (Dense): ")
print(array5)

print("batch_normalization_2 (Batch: ")
print(array6)




conv2d_1 (Conv2D): 
[array([[[[-0.8758111 ,  0.50777876,  0.45769432, -0.17346485,
          -0.51935613, -0.67460626, -0.96310246,  0.27822152,
          -0.9486104 , -0.15331171,  0.7824823 ,  0.3562359 ,
           0.37563854,  0.3546658 , -0.6170942 ,  0.5991493 ,
          -0.62925345,  0.45718983,  0.06285524,  0.41465008,
           0.8373344 ,  0.07167278,  0.4822994 , -0.56259227,
          -0.90744394, -0.9728556 , -0.93681186,  0.6327781 ,
          -0.17852491, -0.05434499, -0.09542767,  0.93598473],
         [ 0.02264589, -0.3836813 , -0.5443037 , -0.8585147 ,
          -0.7698286 ,  0.5968471 , -0.26632315,  0.04070378,
          -0.9993009 ,  0.88129973,  0.07082173,  0.6876599 ,
          -0.9551064 , -0.4949725 ,  0.8342808 , -0.9003771 ,
          -0.1269915 , -0.92089856, -0.6498041 ,  0.749566  ,
           0.04684119, -0.9845775 ,  0.16470583, -0.946684  ,
          -0.42503804,  0.34743315,  0.2674538 ,  0.892276  ,
          -0.10476679, -0.25192243, -0.76748717,

In [3]:
def init_individual():
    print('individual')

init_individual()

individual


In [4]:
def init_population():
    print('population')
    
init_population()

population


In [5]:
def test_population():
    print('testing')

test_population()

testing


In [6]:
def select_best_individuals():
    list = [2,5,342,12,234,1234,23]
    list.sort()
    print(list)

select_best_individuals()

[2, 5, 12, 23, 234, 342, 1234]


In [7]:
def crossover_individuals():
    """
    Slices both dna1 and dna2 into two parts at a random index within their
    length and merges them. Both keep their initial sublist up to the crossover
    index, but their ends are swapped.
    """
    list1 = [1,2,3,4]
    list2 = [11,12,13,14]
    pos=int(random.random()*len(list1))
    print(pos)
    print(list1, list2)
    print(list1[:pos]+list2[pos:], list2[:pos]+list1[pos:])
    
crossover_individuals()

0
[1, 2, 3, 4] [11, 12, 13, 14]
[11, 12, 13, 14] [1, 2, 3, 4]


In [8]:
def random_float():
    """
    Return a random number between -1 and 1
    """
    return random.uniform(-1, 1)

In [9]:

def mutate():
    """
    For each gene in the DNA, there is a 1/mutation_chance chance that it will be
    switched out with a random character. This ensures diversity in the
    population, and ensures that is difficult to get stuck in local minima.
    """
    mutation_chance = 100
    index = 0
    dna_out=[]
    list1 = [1,2,3,4,5,6,7,8,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

    for i in list1:
        if int(random.random()*mutation_chance) == 1:
            dna_out.insert(index, random_float())
        else:
            dna_out.insert(index, i)
        index+1

    print(list(reversed(dna_out)))

mutate()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 1, 1, 1, 1, 1, -0.6734020016622493, 1, 1, 1, 1, 1, 1, 1, -0.5218224671612062, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
def execute_rna(list0, list1, list2, list3, list4, list5, list6, list7):

    dataset = 'dataset'

    EPOCHS = 3
    INIT_LR = 1e-3
    MOMENTUM = 0.8
    BS = 5
    IMAGE_DIMS = (96, 96, 3)
    modelname = 'blood.model'
    label = 'lb.pickle'

    data = []
    labels = []

    imagePaths = list(paths.list_images(dataset))

    for imagePath in imagePaths:
        # carrega a imagem, pré-processa e armazena na lista de dados
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)

        # extrai o rótulo da classe do caminho da imagem e atualiza a lista de rótulos
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    print("[INFO] data matrix: {:.2f}MB".format(
        data.nbytes / (1024 * 1000.0)))

    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)

    (trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.2, shuffle=False )

    aug = ImageDataGenerator()
    width=IMAGE_DIMS[1]
    height=IMAGE_DIMS[0]
    depth=IMAGE_DIMS[2]
    classes=len(lb.classes_)

    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
        chanDim = 1

    # CONV => RELU => POOL
    model.add(Conv2D(32, (3, 3), padding="same",
        input_shape=inputShape, use_bias=True))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    #set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1))
    model.add(BatchNormalization())


    print('original')
    print('#########################################################')
    for layer in model.layers:
        weights = layer.get_weights()
    #     print(weights)

    # layer0 = layer[0].get_weights()
    print(model.layers)
    print(model.get_layer(Conv2D,0).get_weights())
    print(model.get_layer(Conv2D,1).get_weights())
    print(model.get_layer(Conv2D,2).get_weights())
    print(model.get_layer(Conv2D,3).get_weights())
    print(model.get_layer(Conv2D,4).get_weights())
    print(model.get_layer(Conv2D,5).get_weights())
    print(model.get_layer(Conv2D,6).get_weights())
    print(model.get_layer(Conv2D,7).get_weights())

    model.summary()    

    # Classificador
    model.add(Dense(classes))
    model.add(Activation("tanh")) #tanh <- Tangente hiperbólica
    opt = optimizers.SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS, momentum=MOMENTUM, nesterov=True)
    
    model.get_layer(Conv2D,0).set_weights(list0)
    model.get_layer(Conv2D,1).set_weights(list1)
    model.get_layer(Conv2D,2).set_weights(list2)
    model.get_layer(Conv2D,3).set_weights(list3)
    model.get_layer(Conv2D,4).set_weights(list4)
    model.get_layer(Conv2D,5).set_weights(list5)
    model.get_layer(Conv2D,6).set_weights(list6)
    model.get_layer(Conv2D,7).set_weights(list7)
    
    print(model.get_layer(Conv2D,1).get_weights()) 
    print(model.get_layer(Conv2D,1).get_weights())
    print(model.get_layer(Conv2D,2).get_weights())
    print(model.get_layer(Conv2D,3).get_weights())
    print(model.get_layer(Conv2D,4).get_weights())
    print(model.get_layer(Conv2D,5).get_weights())
    print(model.get_layer(Conv2D,6).get_weights())
    print(model.get_layer(Conv2D,7).get_weights())
    
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])
    plot_losses = livelossplot.PlotLossesKeras()
    H = model.fit_generator(
        aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY),
        steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS,
        #callbacks=[plot_losses],
        verbose=1
    )
    train_loss = H.history["loss"]
    val_loss = H.history["val_loss"]
    train_acc = H.history["acc"]
    val_acc = H.history["val_acc"]

    fitness = (sum(train_acc)/len(train_acc))
    return fitness

fit = execute_rna(array00, array0, array1, array2, array3, array4, array5, array6)
print(fit)

[INFO] data matrix: 4.32MB
original
#########################################################
[<keras.layers.convolutional.Conv2D object at 0x000001C760048278>, <keras.layers.core.Activation object at 0x000001C7600484A8>, <keras.layers.normalization.BatchNormalization object at 0x000001C7600483C8>, <keras.layers.pooling.MaxPooling2D object at 0x000001C760048E48>, <keras.layers.core.Flatten object at 0x000001C760053BA8>, <keras.layers.core.Dense object at 0x000001C7600292E8>, <keras.layers.normalization.BatchNormalization object at 0x000001C7610DA470>]
[]
[array([[[[ 4.66841459e-03, -3.78379226e-02, -5.75218946e-02,
           1.33633837e-01, -8.21907222e-02, -6.09466359e-02,
           1.29974380e-01, -5.36828116e-02,  9.64359343e-02,
           1.00378737e-01, -1.04789339e-01,  1.06191352e-01,
          -4.77577299e-02, -7.66143650e-02,  3.53422165e-02,
           2.78816968e-02,  1.21157765e-02,  8.05076659e-02,
           2.29997337e-02, -1.18968159e-01, -1.35662615e-01,
          -

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)]
[]
[]
[array([[ 0.00606156],
       [ 0.00465756],
       [-0.01224928],
       ...,
       [-0.00814995],
       [-0.00028325],
       [-0.00969937]], dtype=float32), array([0.], dtype=float32)]
[array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32)]
______________________________

Epoch 1/3
3/3 [==============================] - 1s 382ms/step - loss: 6.0928 - acc: 0.2000 - val_loss: 0.5057 - val_acc: 0.2500
Epoch 2/3
3/3 [==============================] - 0s 33ms/step - loss: 9.5287 - acc: 0.4182 - val_loss: 0.5058 - val_acc: 0.5000
Epoch 3/3
3/3 [==============================] - 0s 28ms/step - loss: 1.8780 - acc: 0.1455 - val_loss: 0.6051 - val_acc: 0.7500
0.21818182098143027
